In [178]:
import torch
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn, optim
import torch.nn.functional as F

In [179]:
import pandas as pd
import numpy as np
import re
import datetime
import requests
pd.set_option('mode.chained_assignment',None)

In [180]:
# for nba
df = pd.read_pickle('/Users/tonyowens/Downloads/improved_nba_data1.pkl')

In [181]:
df.head()

,Date,Year,Away,Away_Pts,Home,Home_Pts,Home_Team,Home_PPG,Home_FGM,Home_FGA,...,Away_PF,Away_W,Away_L,Away_W/L%,Away_MOV/A,Away_ORtg/A,Away_DRtg/A,Target,Away_b2b,Home_b2b
0,1999-11-12,1999,Golden State,103,Vancouver,93,Vancouver,93.9,35.3,78.5,...,24.9,19,63,0.232,-7.62,100.70,108.65,0,0,0
1,1999-11-12,1999,Utah,92,Sacramento,105,Sacramento,105.0,40.0,88.9,...,24.5,55,27,0.671,4.52,108.00,103.06,1,0,0
2,1999-11-12,1999,Chicago,86,Philadelphia,92,Philadelphia,94.8,36.5,82.6,...,23.3,17,65,0.207,-9.23,95.42,105.65,1,0,0
3,1999-11-12,1999,Phoenix,92,Milwaukee,107,Milwaukee,101.2,38.7,83.3,...,24.1,53,29,0.646,5.24,104.90,99.48,1,0,0
4,1999-11-12,1999,Houston,97,L.A. Lakers,81,L.A. Lakers,100.8,38.3,83.4,...,20.3,34,48,0.415,-0.56,105.72,106.41,0,0,0


In [182]:
all_possible = ['Home_PPG', 'Home_FGM', 'Home_FGA', 'Home_FG%', 'Home_3PM', 'Home_3PA',
       'Home_3P%', 'Home_FTM', 'Home_FTA', 'Home_FT%', 'Home_ORB', 'Home_DRB',
       'Home_APG', 'Home_SPG', 'Home_BPG', 'Home_TOV', 'Home_PF', 'Home_W',
       'Home_L', 'Home_W/L%', 'Home_MOV/A', 'Home_ORtg/A', 'Home_DRtg/A',
                'Away_PPG', 'Away_FGM', 'Away_FGA', 'Away_FG%', 'Away_3PM',
       'Away_3PA', 'Away_3P%', 'Away_FTM', 'Away_FTA', 'Away_FT%', 'Away_ORB',
       'Away_DRB', 'Away_APG', 'Away_SPG', 'Away_BPG', 'Away_TOV', 'Away_PF',
       'Away_W', 'Away_L', 'Away_W/L%', 'Away_MOV/A', 'Away_ORtg/A',
       'Away_DRtg/A','Away_b2b', 'Home_b2b']

In [183]:
normal_cols = all_possible.copy()
normal_cols.remove('Away_b2b')
normal_cols.remove('Home_b2b')

In [184]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(df[normal_cols])

In [185]:
df[normal_cols] = scaler.transform(df[normal_cols])

In [186]:
class MissingDict(dict):
    __missing__ = lambda self, key: key
team_name_conversions = {"New Jersey": "New Jersey Nets", "Houston" : "Houston Rockets", "Utah" : "Utah Jazz", "L.A. Clippers": "Los Angeles Clippers", "L.A. Lakers": "Los Angeles Lakers", "Phoenix": "Phoenix Suns", "Milwaukee" : "Milwaukee Bucks", "Philadelphia": "Philadelphia 76ers", "Denver" : "Denver Nuggets","Brooklyn": "Brooklyn Nets", "Dallas" : "Dallas Mavericks", "New York" : "New York Knicks", "Atlanta" : "Atlanta Hawks", "Portland": "Portland Trail Blazers", "Boston": "Boston Celtics", "Golden State" : "Golden State Warriors", "Memphis" : "Memphis Grizzlies", "Miami": "Miami Heat", "Indiana" : "Indiana Pacers", "New Orleans" : "New Orleans Pelicans", "Toronto" : "Toronto Raptors", "Chicago": "Chicago Bulls", "San Antonio" : "San Antonio Spurs", "Washington" : "Washington Wizards", "Sacramento" : "Sacramento Kings", "Detroit" : "Detroit Pistons", "Minnesota" : "Minnesota Timberwolves", "Charlotte": "Charlotte Bobcats", "Cleveland": "Cleveland Cavaliers", "Orlando": "Orlando Magic", "Oklahoma City" : "Oklahoma City Thunder"}
name_convs = {v: k for k, v in team_name_conversions.items()}
name_convs['New Orleans Hornets'] = "New Orleans"
name_convs['Charlotte Hornets'] = "Charlotte"
name_convs["Seattle SuperSonics"] = "Seattle"
name_convs["Vancouver Grizzlies"] = "Vancouver"
name_convs["New Orleans/Oklahoma City Hornets"] = "New Orleans"
maps = MissingDict(**name_convs)

In [187]:
all_years = []
all_months = []
stats_url = f"https://www.basketball-reference.com/leagues/NBA_{2023}_ratings.html"
data_stats = requests.get(stats_url)
stats = pd.read_html(data_stats.content)[0]
stats.columns = stats.columns.droplevel()
months = ['october', 'november', 'december', 'january', 'february', 'march']
month_abrv = ['december', 'january', 'february', 'march']
month_nov = ['november','december', 'january', 'february', 'march']
ms_url = f"https://basketball.realgm.com/nba/team-stats/{2023}/Averages/Team_Totals/Regular_Season"
ms_data = requests.get(ms_url)
ms_stats = pd.read_html(ms_data.content)[-1]
stats.Team = stats.Team.map(name_convs)
stats = ms_stats.merge(stats,left_on = ["Team"],right_on = ["Team"], how = "outer")

In [188]:
stats_cols = ['PPG', 'FGM', 'FGA', 'FG%', '3PM', '3PA',
       '3P%', 'FTM', 'FTA', 'FT%', 'ORB', 'DRB', 'RPG', 'APG', 'SPG', 'BPG',
       'TOV', 'PF', 'Rk','W', 'L', 'W/L%', 'MOV', 'ORtg',
       'DRtg', 'NRtg', 'MOV/A', 'ORtg/A', 'DRtg/A', 'NRtg/A']

In [189]:
stats[stats_cols] = stats[stats_cols].astype(float)

In [190]:
relevant_cols = ['Date', 'Away', 'Away Pts', 'Home', 'Home Pts',
       'Home Team', 'Home PPG', 'Home FGM', 'Home FGA',
       'Home FG%', 'Home 3PM', 'Home 3PA', 'Home 3P%', 'Home FTM', 'Home FTA',
       'Home FT%', 'Home ORB', 'Home DRB','Home APG', 'Home SPG',
       'Home BPG', 'Home TOV', 'Home PF', 'Home W', 'Home L', 'Home W/L%','Home MOV/A', 
                 'Home ORtg/A', 'Home DRtg/A','Away Team', 
        'Away PPG', 'Away FGM',
       'Away FGA', 'Away FG%', 'Away 3PM', 'Away 3PA', 'Away 3P%', 'Away FTM',
       'Away FTA', 'Away FT%', 'Away ORB', 'Away DRB',  'Away APG',
       'Away SPG', 'Away BPG', 'Away TOV', 'Away PF','Away W', 'Away L', 
                 'Away W/L%','Away MOV/A', 'Away ORtg/A', 'Away DRtg/A']

In [191]:
all_years = []
all_months = []
for month in months:
    url = f"https://www.basketball-reference.com/leagues/NBA_{2023}_games-{month}.html"
    data = requests.get(url)
    sched = pd.read_html(data.content)[0]
    sched = sched.rename(columns = {"Visitor/Neutral" : "Away", "Home/Neutral" : "Home", "PTS" : "Away Pts", "PTS.1" : "Home Pts"})
    attempt = []
    for j in sched.index:
        test = sched.loc[sched.index == j]
        test.Home = test.Home.map(name_convs)
        test.Away = test.Away.map(name_convs)
        attempt.append(pd.merge(pd.merge(test,stats[stats['Team'].isin(test['Home'])].add_prefix('Home '),left_on='Home',right_on = 'Home Team'),stats[stats['Team'].isin(test['Away'])].add_prefix('Away '),left_on = 'Away', right_on = 'Away Team'))
    all_months.append(pd.concat(attempt))
    full_sched = pd.concat(all_months).sort_values(by ='Date').reset_index(drop = True)
    all_years.append(full_sched)
current_sched = pd.concat(all_years)[relevant_cols]

In [192]:
format_cols = []
for i in relevant_cols:
    i = i.replace(' ','_')
    format_cols.append(i)

In [193]:
current_sched.columns = format_cols

In [194]:
current_sched.Date = current_sched.Date.apply(lambda x: re.split('[A-Z]{1}[a-z]{2}, ',x)[1])

In [195]:
current_sched.Date = pd.to_datetime(current_sched.Date)

In [196]:
def curr_b2b_home(row):
    prev_row = current_sched[current_sched.Date == row.Date - datetime.timedelta(days = 1)]
    if len(prev_row) == 0:
        return 0
    elif(len(prev_row[(prev_row.Home == row.Home) | (prev_row.Away == row.Home)]) > 0):
        return 1
    return 0
def curr_b2b_away(row):
    prev_row = current_sched[current_sched.Date == row.Date - datetime.timedelta(days = 1)]
    if len(prev_row) == 0:
        return 0
    elif(len(prev_row[(prev_row.Home == row.Away) | (prev_row.Away == row.Away)]) > 0):
        return 1
    return 0

In [197]:
current_sched['Away_b2b'] = current_sched.apply(lambda x:curr_b2b_away(x),axis = 1)
current_sched['Home_b2b'] = current_sched.apply(lambda x:curr_b2b_home(x),axis = 1)

In [198]:
X,y = df[all_possible],df.Target

In [199]:
from sklearn.model_selection import train_test_split
# X_train,X_test,y_train,y_test = train_test_split(X, y,train_size=.95)
X_non_test, X_test, y_non_test,y_test = train_test_split(X, 
                                                           y,train_size=.80)
X_train, X_val, y_train,y_val = train_test_split(X_non_test, 
                                                           y_non_test,train_size=.80)

In [200]:
X_train = torch.from_numpy(X_train.to_numpy()).float()
y_train = torch.squeeze(torch.from_numpy(y_train.to_numpy()).float())
X_val = torch.squeeze(torch.from_numpy(X_val.to_numpy()).float())
y_val = torch.from_numpy(y_val.to_numpy()).float()
X_test = torch.from_numpy(X_test.to_numpy()).float()
y_test = torch.squeeze(torch.from_numpy(y_test.to_numpy()).float())

In [201]:
class Net(nn.Module):
  def __init__(self, n_features):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(n_features, 80)
    self.fc2 = nn.Linear(80, 100)
    self.fc3 = nn.Linear(100, 150)
    self.fc4 = nn.Linear(150, 300)
    self.fc5 = nn.Linear(300, 1)
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    return torch.sigmoid(self.fc5(x))
net = Net(X_train.shape[1])

In [202]:
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=1e-2)

In [203]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
net = net.to(device)
criterion = criterion.to(device)

In [204]:
def calculate_accuracy(y_true, y_pred):
  predicted = y_pred.ge(.5).view(-1)
  return (y_true == predicted).sum().float() / len(y_true)


In [205]:
def round_tensor(t, decimal_places=3):
  return round(t.item(), decimal_places)
for epoch in range(5001):
    y_pred = net(X_train)
    y_pred = torch.squeeze(y_pred)
    train_loss = criterion(y_pred, y_train)
    if epoch % 1000 == 0:
      train_acc = calculate_accuracy(y_train, y_pred)
      y_test_pred = net(X_val)
      y_test_pred = torch.squeeze(y_test_pred)
      test_loss = criterion(y_test_pred, y_val)
      test_acc = calculate_accuracy(y_val, y_test_pred)
      print(
f'''epoch {epoch}
Train set - loss: {round_tensor(train_loss)}, accuracy: {100*round_tensor(train_acc,4)}%
Test  set - loss: {round_tensor(test_loss)}, accuracy: {100*round_tensor(test_acc,4)}%
''')
#     optimizer.zero_grad()
    for param in net.parameters():   
        param.grad = None
    train_loss.backward()
    optimizer.step()

epoch 0
Train set - loss: 0.695, accuracy: 40.61%
Test  set - loss: 0.695, accuracy: 40.12%

epoch 1000
Train set - loss: 0.644, accuracy: 59.41%
Test  set - loss: 0.644, accuracy: 59.88%

epoch 2000
Train set - loss: 0.581, accuracy: 69.19999999999999%
Test  set - loss: 0.588, accuracy: 68.24%

epoch 3000
Train set - loss: 0.575, accuracy: 69.46%
Test  set - loss: 0.584, accuracy: 68.58%

epoch 4000
Train set - loss: 0.573, accuracy: 69.47%
Test  set - loss: 0.583, accuracy: 69.0%

epoch 5000
Train set - loss: 0.571, accuracy: 69.77%
Test  set - loss: 0.582, accuracy: 69.08%



# Evaluate based on the test set 

In [206]:
preds1 = np.round(net(X_test).detach().numpy())

In [207]:
b = []
for y1 in y_test.detach().numpy():
    b = np.append(b,y1)

In [208]:
a = []
for pred in preds1:
    a = np.append(a,pred)
    

In [209]:
from sklearn.metrics import accuracy_score
accq = accuracy_score(b,a)
print(accuracy_score(b,a))
combinedq = pd.DataFrame(dict(actual =b ,prediction = a))
pd.crosstab(index=combinedq["actual"], columns = combinedq["prediction"])

0.6917394224311618


prediction,0.0,1.0
actual,,
0.0,982,793
1.0,584,2108


In [210]:
print("Accuracy when predicting home win = %.3f" % (np.sum((combinedq.actual == combinedq.prediction) & (combinedq.prediction == 1)) / np.sum(combinedq.prediction)))

Accuracy when predicting home win = 0.727


In [211]:
print("Accuracy when predicting road win = %.3f" % 
     (np.sum((combinedq.actual == combinedq.prediction) & (combinedq.prediction == 0))
      / np.sum(combinedq.prediction == 0)))

Accuracy when predicting road win = 0.627


### It is clear that the accuracy is much higher when predicting the home team to win than predicting the away team. My first instinct is that this is caused by a data imbalanced due to the fact that in the NBA,as in most sports, the home team wins the majority of the time

In [212]:
print("The home team wins %.1f%% percent of the time" % (100 * sum(y) / len(y)))

The home team wins 59.7% percent of the time


# It might be beneficial to balance the dataset
### This will be done in R to take advantage of the ROSE package

In [213]:
# convert to a csv to use in R
df[all_possible + ['Target']].to_csv('/Users/tonyowens/Downloads/nba_raw_data.csv')

# With simulated data

In [214]:
nba_simul_data = pd.read_csv('/Users/tonyowens/Downloads/nba_simulated_data.csv',index_col = [0])

In [215]:
nba_cols = ['Home_PPG', 'Home_FGM', 'Home_FGA', 'Home_FG.', 'Home_3PM', 'Home_3PA',
       'Home_3P.', 'Home_FTM', 'Home_FTA', 'Home_FT.', 'Home_ORB', 'Home_DRB',
       'Home_APG', 'Home_SPG', 'Home_BPG', 'Home_TOV', 'Home_PF', 'Home_W',
       'Home_L', 'Home_W.L.', 'Home_MOV.A', 'Home_ORtg.A', 'Home_DRtg.A',
       'Away_PPG', 'Away_FGM', 'Away_FGA', 'Away_FG.', 'Away_3PM', 'Away_3PA',
       'Away_3P.', 'Away_FTM', 'Away_FTA', 'Away_FT.', 'Away_ORB', 'Away_DRB',
       'Away_APG', 'Away_SPG', 'Away_BPG', 'Away_TOV', 'Away_PF', 'Away_W',
       'Away_L', 'Away_W.L.', 'Away_MOV.A', 'Away_ORtg.A', 'Away_DRtg.A',
       'Away_b2b', 'Home_b2b']

In [216]:
X_new,y_new = nba_simul_data[nba_cols],nba_simul_data.Target

In [217]:
print("The home team now wins %.1f%% percent of the time" % (100 * sum(y_new) / len(y_new)))

The home team now wins 50.1% percent of the time


## This is much more balanced and should reduce the gap between home and away accuracy

In [218]:
from sklearn.model_selection import train_test_split
# X_train,X_test,y_train,y_test = train_test_split(X, y,train_size=.95)
X_non_test, X_test, y_non_test,y_test = train_test_split(X_new, 
                                                           y_new,train_size=.80)
X_train, X_val, y_train,y_val = train_test_split(X_non_test, 
                                                           y_non_test,train_size=.80)

In [219]:
X_train = torch.from_numpy(X_train.to_numpy()).float()
y_train = torch.squeeze(torch.from_numpy(y_train.to_numpy()).float())
X_val = torch.squeeze(torch.from_numpy(X_val.to_numpy()).float())
y_val = torch.from_numpy(y_val.to_numpy()).float()
X_test = torch.from_numpy(X_test.to_numpy()).float()
y_test = torch.squeeze(torch.from_numpy(y_test.to_numpy()).float())

In [220]:
class Net(nn.Module):
  def __init__(self, n_features):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(n_features, 80)
    self.fc2 = nn.Linear(80, 100)
    self.fc3 = nn.Linear(100, 150)
    self.fc4 = nn.Linear(150, 350)
    self.fc5 = nn.Linear(350, 1)
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    return torch.sigmoid(self.fc5(x))
net = Net(X_train.shape[1])

In [221]:
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=1e-2)

In [222]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
net = net.to(device)
criterion = criterion.to(device)

In [223]:
def calculate_accuracy(y_true, y_pred):
  predicted = y_pred.ge(.5).view(-1)
  return (y_true == predicted).sum().float() / len(y_true)


In [224]:
#tryna beat 68.7 #almost guaranteed that it will be defeated

In [225]:
def round_tensor(t, decimal_places=3):
  return round(t.item(), decimal_places)
for epoch in range(10001):
    y_pred = net(X_train)
    y_pred = torch.squeeze(y_pred)
    train_loss = criterion(y_pred, y_train)
    if epoch % 2000 == 0:
      train_acc = calculate_accuracy(y_train, y_pred)
      y_test_pred = net(X_val)
      y_test_pred = torch.squeeze(y_test_pred)
      test_loss = criterion(y_test_pred, y_val)
      test_acc = calculate_accuracy(y_val, y_test_pred)
      print(
f'''epoch {epoch}
Train set - loss: {round_tensor(train_loss)}, accuracy: {100*round_tensor(train_acc,4)}%
Test  set - loss: {round_tensor(test_loss)}, accuracy: {100*round_tensor(test_acc,4)}%
''')
#     optimizer.zero_grad()
    for param in net.parameters():   
        param.grad = None
    train_loss.backward()
    optimizer.step()

epoch 0
Train set - loss: 0.693, accuracy: 49.220000000000006%
Test  set - loss: 0.693, accuracy: 49.44%

epoch 2000
Train set - loss: 0.609, accuracy: 67.66%
Test  set - loss: 0.61, accuracy: 67.35%

epoch 4000
Train set - loss: 0.591, accuracy: 68.58%
Test  set - loss: 0.593, accuracy: 68.30000000000001%

epoch 6000
Train set - loss: 0.587, accuracy: 68.84%
Test  set - loss: 0.589, accuracy: 68.66%

epoch 8000
Train set - loss: 0.584, accuracy: 69.17%
Test  set - loss: 0.586, accuracy: 69.04%

epoch 10000
Train set - loss: 0.582, accuracy: 69.19999999999999%
Test  set - loss: 0.584, accuracy: 69.12%



In [226]:
preds_simul = np.round(net(X_test).detach().numpy())

In [227]:
b = []
for y1 in y_test.detach().numpy():
    b = np.append(b,y1)

In [228]:
a = []
for pred in preds_simul:
    a = np.append(a,pred)
    

In [229]:
from sklearn.metrics import accuracy_score
accq = accuracy_score(b,a)
print(accuracy_score(b,a))
combinedq = pd.DataFrame(dict(actual =b ,prediction = a))
pd.crosstab(index=combinedq["actual"], columns = combinedq["prediction"])

0.69635


prediction,0.0,1.0
actual,,
0.0,7198,2839
1.0,3234,6729


In [230]:
print("Accuracy when predicting home win = %.3f" % (np.sum((combinedq.actual == combinedq.prediction) & (combinedq.prediction == 1))
                                                    / np.sum(combinedq.prediction)))

Accuracy when predicting home win = 0.703


In [231]:
print("Accuracy when predicting road win = %.3f" % 
     (np.sum((combinedq.actual == combinedq.prediction) & (combinedq.prediction == 0))
      / np.sum(combinedq.prediction == 0)))

Accuracy when predicting road win = 0.690


### While a gap still exists, I would attribute the majority of the gap due to the fact that the away team winning likely features more upsets which are inherently harder to predict than the favored team winning.

# Predicting games on a given day

In [232]:
today = datetime.datetime.today().strftime("%b %d, %Y")
today

'Nov 18, 2022'

In [233]:
todays_fixtures = current_sched[current_sched.Date == today]
todays_fixtures.drop(columns = ['Away_Pts','Home_Pts'],inplace = True)
todays_fixtures.drop_duplicates(inplace=True)
todays_fixtures.reset_index(drop = True,inplace = True)
todays_fixtures.head()

,Date,Away,Home,Home_Team,Home_PPG,Home_FGM,Home_FGA,Home_FG%,Home_3PM,Home_3PA,...,Away_TOV,Away_PF,Away_W,Away_L,Away_W/L%,Away_MOV/A,Away_ORtg/A,Away_DRtg/A,Away_b2b,Home_b2b
0,2022-11-18,Phoenix,Utah,Utah,116.6,42.6,91.5,0.465,14.7,40.0,...,13.3,21.6,9.0,5.0,0.643,6.84,118.66,111.82,0,0
1,2022-11-18,Boston,New Orleans,New Orleans,116.7,43.6,90.7,0.481,11.1,28.9,...,13.0,21.2,12.0,3.0,0.800,6.92,121.02,113.87,0,0
2,2022-11-18,Denver,Dallas,Dallas,108.6,37.9,83.8,0.452,13.5,39.9,...,15.6,18.6,9.0,5.0,0.643,0.93,115.70,114.91,0,0
3,2022-11-18,Charlotte,Cleveland,Cleveland,114.7,41.0,87.1,0.470,13.0,32.4,...,14.9,21.5,4.0,12.0,0.250,-5.80,107.87,113.64,0,0
4,2022-11-18,Orlando,Chicago,Chicago,110.7,40.4,86.3,0.468,10.4,29.0,...,16.9,21.7,4.0,11.0,0.267,-3.58,110.38,113.97,0,0


In [234]:
# day_preds = pd.read_pickle(f'/Users/tonyowens/Downloads/NN_picks_for{today}.pkl')
day_preds = todays_fixtures[all_possible].copy()

In [235]:
day_preds[normal_cols] = pd.DataFrame(scaler.transform(day_preds[normal_cols]))
day_preds.columns = X.columns
day_preds = torch.from_numpy(day_preds.to_numpy()).float()

In [236]:
# predict the games using the neural network
preds = net(day_preds)

In [237]:
todays_fixtures['prob'] = np.round(preds.detach().numpy(),2)

In [238]:
todays_fixtures['winner'] = todays_fixtures.apply(lambda x:x['Home'] if x['prob'] >.5 else x['Away'],axis = 1)

In [239]:
def odds_formula(percent):
    if percent > .5:
        val =  100 + 100 / (percent - 1)
        return f"-{int(abs(val))}"
    else:
        val = 100 / percent - 100
        return f"-{int(abs(val))}"

In [240]:
# Convert the probabilities into American betting odds.
todays_fixtures['Winner Line'] = todays_fixtures['prob'].apply(lambda x: odds_formula(x))

In [241]:
todays_fixtures[['Home','Away','prob','winner','Winner Line']]

,Home,Away,prob,winner,Winner Line
0,Utah,Phoenix,0.41,Phoenix,-143
1,New Orleans,Boston,0.36,Boston,-177
2,Dallas,Denver,0.42,Denver,-138
3,Cleveland,Charlotte,0.71,Cleveland,-244
4,Chicago,Orlando,0.47,Orlando,-112
5,Philadelphia,Milwaukee,0.31,Milwaukee,-222
6,Golden State,New York,0.31,New York,-222
7,Washington,Miami,0.40,Miami,-149
8,Houston,Indiana,0.23,Indiana,-334
9,L.A. Lakers,Detroit,0.40,Detroit,-149
